In [1]:
import pandas as pd
import numpy as np
import torch
import networkx as nx
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

## 1. Tiền xử lý dữ liệu

In [2]:
# Đọc từng file từ thư mục MXH_Dataset
train_df = pd.read_csv("../Dataset/train.csv")
segment_status_df = pd.read_csv("../Dataset/segment_status.csv")


In [3]:
# Kiểm tra số dòng, số cột của từng file
for name, df in [("segment_status", segment_status_df), 
                 ("train", train_df)]:
    print(f"{name}: {df.info}")

segment_status: <bound method DataFrame.info of          _id                updated_at  segment_id  velocity
0          0  2020-07-03T14:55:31.869Z       24845        20
1          1  2020-07-03T15:02:56.048Z       33923        10
2          2  2020-07-04T08:15:52.696Z       33824         5
3          3  2020-07-04T08:15:59.903Z       33824         5
4          4  2020-07-04T08:16:08.201Z       33824         5
...      ...                       ...         ...       ...
90933  90933  2021-04-22T06:52:39.280Z       52247         1
90934  90934  2021-04-22T06:52:52.501Z       52247         1
90935  90935  2021-04-22T06:53:02.335Z       52247         1
90936  90936  2021-04-22T06:53:14.294Z       52247         1
90937  90937  2021-04-22T06:53:27.300Z       52247         1

[90938 rows x 4 columns]>
train: <bound method DataFrame.info of          _id  segment_id        date  weekday        period LOS   s_node_id  \
0          0          26  2021-04-16        4   period_0_30   A   366428456

### 1.1 Xử lý dữ liệu thiếu

In [4]:
print("Missing values in train_df:\n", train_df.isnull().sum())

Missing values in train_df:
 _id                 0
segment_id          0
date                0
weekday             0
period              0
LOS                 0
s_node_id           0
e_node_id           0
length              0
street_id           0
max_velocity    28495
street_level        0
street_name         1
street_type         0
long_snode          0
lat_snode           0
long_enode          0
lat_enode           0
dtype: int64


In [5]:
# Xử lý giá trị thiếu
train_df['max_velocity'] = train_df['max_velocity'].fillna(train_df['max_velocity'].mean())
train_df['street_name'] = train_df['street_name'].fillna('Unknown')

print("Missing values in train_df:\n", train_df.isnull().sum())


Missing values in train_df:
 _id             0
segment_id      0
date            0
weekday         0
period          0
LOS             0
s_node_id       0
e_node_id       0
length          0
street_id       0
max_velocity    0
street_level    0
street_name     0
street_type     0
long_snode      0
lat_snode       0
long_enode      0
lat_enode       0
dtype: int64


In [6]:
# Chuyển đổi thời gian
train_df['date'] = pd.to_datetime(train_df['date'])
segment_status_df['updated_at'] = pd.to_datetime(segment_status_df['updated_at'])
train_df = train_df.sort_values('date')

### 1.2 Chuẩn hóa cột LOS

In [7]:
le = LabelEncoder()
train_df['LOS_encoded'] = le.fit_transform(train_df['LOS'])
scaler = MinMaxScaler()
train_df['LOS_norm'] = scaler.fit_transform(train_df[['LOS_encoded']])

### 1.3  Tạo đặc trưng nút

In [15]:
edge_index = train_df[['s_node_id', 'e_node_id', 'segment_id']].drop_duplicates().dropna().astype(int)
# Trước khi tạo edge_index_torch
node_ids = sorted(set(edge_index['s_node_id']).union(edge_index['e_node_id']))
node_id_to_index = {node_id: idx for idx, node_id in enumerate(node_ids)}
num_nodes = len(node_ids)

# Cập nhật edge_index với chỉ số mới
edge_index['s_node_idx'] = edge_index['s_node_id'].map(node_id_to_index)
edge_index['e_node_idx'] = edge_index['e_node_id'].map(node_id_to_index)
edge_index_torch = torch.tensor(edge_index[['s_node_idx', 'e_node_idx']].T.values, dtype=torch.int64)

# Tính số nút
num_nodes = len(set(edge_index['s_node_id']).union(edge_index['e_node_id']))

# Tạo đặc trưng nút
node_degrees = pd.concat([
    edge_index['s_node_id'].value_counts(),
    edge_index['e_node_id'].value_counts()
]).groupby(level=0).sum().reindex(range(num_nodes), fill_value=0).values

node_features = []
for node_id in range(num_nodes):
    connected_edges = edge_index[
        (edge_index['s_node_id'] == node_id) | (edge_index['e_node_id'] == node_id)
    ]
    segment_ids = connected_edges['segment_id']
    if len(segment_ids) > 0:
        segment_data = train_df[train_df['segment_id'].isin(segment_ids)]
        avg_length = segment_data['length'].mean() if 'length' in segment_data else 0
        avg_velocity = segment_data['velocity'].mean() if 'velocity' in segment_data else 0
    else:
        avg_length, avg_velocity = 0, 0
    node_features.append([node_degrees[node_id], avg_length, avg_velocity])

node_features = torch.tensor(node_features, dtype=torch.float32)

### 1.4 Merge 2 file segment_status và train

In [9]:
# Hàm convert period (vd: "period_14_30") thành timedelta
def period_to_time(period_str):
    try:
        _, hour_str, min_str = period_str.split("_")
        hour = int(hour_str)
        minute = int(min_str)
        return pd.to_timedelta(f"{hour}:{minute}:00")
    except:
        return pd.NaT

# Apply và tạo cột thời gian đầy đủ
train_df['time_delta'] = train_df['period'].apply(period_to_time)
train_df['date'] = pd.to_datetime(train_df['date']) + train_df['time_delta']

# Xoá cột phụ nếu muốn
train_df.drop(columns='time_delta', inplace=True)

# Chuyển 'date' và 'updated_at' về datetime
train_df['date'] = pd.to_datetime(train_df['date']).dt.tz_localize(None)
segment_status_df['updated_at'] = pd.to_datetime(segment_status_df['updated_at']).dt.tz_localize(None)

# Sort trước khi dùng merge_asof
train = train_df.sort_values(by='date')
segment_status = segment_status_df.sort_values(by='updated_at')

# Merge gần đúng theo thời gian, trong cùng segment_id
merged_df = pd.merge_asof(
    train,
    segment_status,
    by='segment_id',
    left_on='date',
    right_on='updated_at',
    direction='nearest',  # hoặc 'backward' nếu bạn chỉ muốn dùng dữ liệu trước đó
    tolerance=pd.Timedelta('30min')  # chỉ chấp nhận khớp nếu lệch thời gian <= 30 phút
)


### 1.5 Tạo pivot table cho LOS_norm và velocity

In [16]:
# Tạo los_pivot
los_pivot = train_df.pivot_table(
    index='segment_id',
    columns='date',
    values='LOS_norm',
    aggfunc='mean'
).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).sort_index(axis=1)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17560\865394265.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).sort_index(axis=1)


### 1.6 Tạo danh sách data object

In [ ]:
from torch_geometric.data import Data, DataLoader
# Tạo danh sách Data objects
data_list = []
for date in los_pivot.columns:
    edge_features = []
    targets = []
    for _, row in edge_index.iterrows():
        segment_id = row['segment_id']
        if segment_id in los_pivot.index:
            los = los_pivot.loc[segment_id, date]
            edge_features.append([los])
            targets.append([los])
        else:
            edge_features.append([0])
            targets.append([0])
    edge_features = torch.tensor(edge_features, dtype=torch.float32)
    targets = torch.tensor(targets, dtype=torch.float32)
    data = Data(
        x=node_features,
        edge_index=edge_index_torch,
        edge_attr=edge_features,
        y=targets
    )
    data_list.append(data)

# Tạo DataLoader
loader = DataLoader(data_list, batch_size=4, shuffle=False)

# Kiểm tra DataLoader
for data in loader:
    print(f"Data: {data}")
    break

Data: DataBatch(x=[11314, 3], edge_index=[2, 10027], edge_attr=[10027, 1], y=[10027, 1], batch=[11314], ptr=[2])


C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## 2. Định nghĩa mô hình GRNN

In [20]:
from torch_geometric.nn import GCNConv
import torch.nn as nn

class GRNN(nn.Module):
    def __init__(self, node_features_dim, hidden_dim, out_dim):
        super(GRNN, self).__init__()
        self.gcn = GCNConv(node_features_dim, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, out_dim)

    def forward(self, data_list):
        outputs = []
        h = None
        for data in data_list:
            x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
            x = torch.relu(self.gcn(x, edge_index, edge_attr))
            x = x.unsqueeze(0)  # [1, num_nodes, hidden_dim]
            x, h = self.rnn(x, h)
            x = x.squeeze(0)  # [num_nodes, hidden_dim]
            out = self.fc(x)  # [num_nodes, out_dim]
            outputs.append(out[data.edge_index[0]])  # Dự đoán cho cạnh
        return outputs

# Khởi tạo mô hình
model = GRNN(node_features_dim=3, hidden_dim=32, out_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

## 3. Huấn luyện mô hình

In [21]:
# Huấn luyện
model.train()
for epoch in range(20):
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        outputs = model([data])
        loss = criterion(outputs[0], data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(loader)}")

Epoch 1, Loss: 0.12195290588453168
Epoch 2, Loss: 0.11983358572985305


KeyboardInterrupt: 

In [ ]:
# Dự đoán
model.eval()
predictions = []
with torch.no_grad():
    for data in loader:
        outputs = model([data])
        predictions.append(outputs[0].cpu().numpy())

In [ ]:
# Đánh giá
targets = [data.y.cpu().numpy() for data in loader]
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(np.concatenate(targets), np.concatenate(predictions))
print(f"Test MSE: {mse}")

### 1.6 Dùng LOS làm output

In [ ]:
# Kiểm tra kích thước của từng phần tử trong edge_features_sequence
for i, edge_features in enumerate(edge_features_sequence):
    print(f"Snapshot {i}: {edge_features.shape}")

# Kiểm tra kích thước của edge_index_torch
print(f"Edge index shape: {edge_index_torch.shape}")

Snapshot 0: (10027, 1)
Snapshot 1: (10027, 1)
Snapshot 2: (10027, 1)
Snapshot 3: (10027, 1)
Snapshot 4: (10027, 1)
Snapshot 5: (10027, 1)
Snapshot 6: (10027, 1)
Snapshot 7: (10027, 1)
Snapshot 8: (10027, 1)
Snapshot 9: (10027, 1)
Snapshot 10: (10027, 1)
Snapshot 11: (10027, 1)
Snapshot 12: (10027, 1)
Snapshot 13: (10027, 1)
Snapshot 14: (10027, 1)
Snapshot 15: (10027, 1)
Snapshot 16: (10027, 1)
Snapshot 17: (10027, 1)
Snapshot 18: (10027, 1)
Snapshot 19: (10027, 1)
Snapshot 20: (10027, 1)
Snapshot 21: (10027, 1)
Snapshot 22: (10027, 1)
Snapshot 23: (10027, 1)
Snapshot 24: (10027, 1)
Snapshot 25: (10027, 1)
Snapshot 26: (10027, 1)
Snapshot 27: (10027, 1)
Snapshot 28: (10027, 1)
Snapshot 29: (10027, 1)
Snapshot 30: (10027, 1)
Snapshot 31: (10027, 1)
Snapshot 32: (10027, 1)
Snapshot 33: (10027, 1)
Snapshot 34: (10027, 1)
Snapshot 35: (10027, 1)
Snapshot 36: (10027, 1)
Snapshot 37: (10027, 1)
Snapshot 38: (10027, 1)
Snapshot 39: (10027, 1)
Snapshot 40: (10027, 1)
Snapshot 41: (10027, 1)
Sn

In [ ]:
# Kiểm tra dữ liệu đầu vào
print("LOS Pivot shape:", los_pivot.shape)
print("Edge Index DataFrame:")
print(edge_index.head())

LOS Pivot shape: (10027, 122)
Edge Index DataFrame:
        s_node_id   e_node_id  segment_id
13056  5769406275  5769406276       33923
9598   2409410635  3771416347       24845
23197  5772558121  5772558074       56816
12978  2233730990  2299409160       33824
5508    366383168  5243849435       15412


In [ ]:
# Chuyển đổi edge_features_sequence thành danh sách các tensor
edge_features_sequence = [
    torch.tensor(edge_features, dtype=torch.float32) for edge_features in edge_features_sequence
]

# Tạo dataset DynamicGraphTemporalSignal
dataset = DynamicGraphTemporalSignal(
    edge_indices=[edge_index_torch] * len(edge_features_sequence),
    edge_weights=edge_features_sequence,
    features=[torch.tensor(node_features, dtype=torch.float32)] * len(edge_features_sequence),
    targets=edge_features_sequence  # Dùng edge_features_sequence làm output
)

# Kiểm tra snapshot
snapshot = next(iter(dataset))
print("Edge Index:\n", snapshot.edge_index)
print("Edge Weight:\n", snapshot.edge_weight)
print("Node Features:\n", snapshot.x)
print("Targets:\n", snapshot.y)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7052\1155704792.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features=[torch.tensor(node_features, dtype=torch.float32)] * len(edge_features_sequence),


AttributeError: 'torch.dtype' object has no attribute 'kind'

In [ ]:
print(edge_index.isnull().sum())  # Kiểm tra giá trị thiếu
print(set(edge_index['segment_id']) - set(los_pivot.index))  # Kiểm tra segment_id không khớp

s_node_id     0
e_node_id     0
segment_id    0
dtype: int64
set()


In [ ]:
for i, edge_features in enumerate(edge_features_sequence):
    if not isinstance(edge_features, torch.Tensor):
        print(f"Snapshot {i} is not a tensor.")
    if torch.isnan(edge_features).any():
        print(f"Snapshot {i} contains NaN values.")

In [ ]:
print(len(edge_features_sequence))  # Số lượng snapshot
print(len([edge_index_torch] * len(edge_features_sequence)))  # Số lượng edge_indices
print(len([torch.tensor(node_features, dtype=torch.float32)] * len(edge_features_sequence)))  # Số lượng node features

122
122
122


C:\Users\ASUS\AppData\Local\Temp\ipykernel_7052\107659669.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(len([torch.tensor(node_features, dtype=torch.float32)] * len(edge_features_sequence)))  # Số lượng node features


In [ ]:
missing_segments = set(edge_index['segment_id']) - set(los_pivot.index)
print(f"Missing segment IDs: {missing_segments}")

Missing segment IDs: set()


In [ ]:
print(f"Number of snapshots: {len(edge_features_sequence)}")
print(f"Number of edge indices: {len([edge_index_torch] * len(edge_features_sequence))}")
print(f"Number of node features: {len([torch.tensor(node_features, dtype=torch.float32)] * len(edge_features_sequence))}")

Number of snapshots: 122
Number of edge indices: 122
Number of node features: 122


C:\Users\ASUS\AppData\Local\Temp\ipykernel_7052\2850595234.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(f"Number of node features: {len([torch.tensor(node_features, dtype=torch.float32)] * len(edge_features_sequence))}")


In [ ]:
print(los_pivot.head())
print(edge_index.head())

date        2020-07-03  2020-07-04  2020-07-06  2020-07-11  2020-07-12  \
segment_id                                                               
26                 0.0         0.0         0.0         0.0         0.0   
33                 0.4         0.4         0.4         0.4         0.4   
67                 0.2         0.2         0.2         0.2         0.2   
70                 0.6         0.6         0.6         0.6         0.6   
71                 0.8         0.8         0.8         0.8         0.8   

date        2020-07-13  2020-07-15  2020-07-20  2020-07-25  2020-07-29  ...  \
segment_id                                                              ...   
26                 0.0         0.0         0.0         0.0         0.0  ...   
33                 0.4         0.4         0.4         0.4         0.4  ...   
67                 0.2         0.2         0.2         0.2         0.2  ...   
70                 0.6         0.6         0.6         0.6         0.6  ...   
71     

In [ ]:
for i, edge_features in enumerate(edge_features_sequence):
    print(f"Snapshot {i} type: {type(edge_features)}")
    print(f"Snapshot {i} dtype: {edge_features.dtype}")

Snapshot 0 type: <class 'torch.Tensor'>
Snapshot 0 dtype: torch.float32
Snapshot 1 type: <class 'torch.Tensor'>
Snapshot 1 dtype: torch.float32
Snapshot 2 type: <class 'torch.Tensor'>
Snapshot 2 dtype: torch.float32
Snapshot 3 type: <class 'torch.Tensor'>
Snapshot 3 dtype: torch.float32
Snapshot 4 type: <class 'torch.Tensor'>
Snapshot 4 dtype: torch.float32
Snapshot 5 type: <class 'torch.Tensor'>
Snapshot 5 dtype: torch.float32
Snapshot 6 type: <class 'torch.Tensor'>
Snapshot 6 dtype: torch.float32
Snapshot 7 type: <class 'torch.Tensor'>
Snapshot 7 dtype: torch.float32
Snapshot 8 type: <class 'torch.Tensor'>
Snapshot 8 dtype: torch.float32
Snapshot 9 type: <class 'torch.Tensor'>
Snapshot 9 dtype: torch.float32
Snapshot 10 type: <class 'torch.Tensor'>
Snapshot 10 dtype: torch.float32
Snapshot 11 type: <class 'torch.Tensor'>
Snapshot 11 dtype: torch.float32
Snapshot 12 type: <class 'torch.Tensor'>
Snapshot 12 dtype: torch.float32
Snapshot 13 type: <class 'torch.Tensor'>
Snapshot 13 dtype

In [ ]:
print(f"Edge index type: {type(edge_index_torch)}")
print(f"Edge index dtype: {edge_index_torch.dtype}")

Edge index type: <class 'torch.Tensor'>
Edge index dtype: torch.int64


In [ ]:
print(f"Node features type: {type(node_features)}")
if isinstance(node_features, np.ndarray):
    print(f"Node features dtype: {node_features.dtype}")

Node features type: <class 'torch.Tensor'>


In [ ]:
print(edge_features_sequence[0].shape)
print(edge_features_sequence[0].dtype)

torch.Size([10027, 1])
torch.float32


In [ ]:
import torch
import torch_geometric_temporal
print(torch.__version__)
print(torch_geometric_temporal.__version__)

2.0.0+cpu
0.54.0
